In [100]:
from requests import get
from requests.utils import quote
import json
from datetime import datetime
from time import sleep

Helpers

In [ ]:
def exponential_backoff(api_method, backoff_rate=1.5, max_calls=3) -> function:
    '''
    Wrapper for methods that call APIs and return response objects with a ".ok"
    attribute to signify a successful or failed API call

    Args:
        api_method (function):           the function that calls the API endpoint
        backoff_rate (int/float):    multiplier for the exponential backoff rate
                                     (higher means longer)
        max_calls (int):             number of calls before the process ceases attempts
                                     and produces an exception

    Returns:
        wrap (function): 
    '''
    def wrap(*args, **kwargs):

        for _ in range(max_calls):
            # Call Original Function
            api_response = api_method(*args, **kwargs)
            if not api_response.ok:
                sleep_time = backoff_rate
                sleep(sleep_time)
                sleep_time *= backoff_rate
            else:
                return api_response
            
        exception_message = f'Process exceeded retry limit of {max_calls} after {sleep_time} seconds between attempts'
        raise Exception(exception_message)

    return wrap

NameError: name 'function' is not defined

In [ ]:
class WmataClient():
    '''
    Client for the WMATA
    '''
    def __init__(self, wmata_api_token):
        self.basepath = 'http://api.wmata.com/'
        self.wmata_api_token = wmata_api_token
        self.request_timeout_seconds = 10

    @exponential_backoff
    def __api_call(self, method_url):
        '''

        Args:
            method_url (str): url portion to be added to the wmata basepath
        '''
        full_url = self.basepath + method_url
        headers = {'api_key': self.wmata_api_token}
        api_response = get(full_url, headers=headers,
                           timeout=self.request_timeout_seconds)
        
        return api_response
    
    def get_prediction(self, station_codes):
        '''
        Details: https://developer.wmata.com/api-details#api=547636a6f9182302184cda78&operation=547636a6f918230da855363f

        - Returns next train arrival information for one or more stations
        - Use "All" for the StationCodes parameter to return predictions for all stations
        - Next train arrival information is refreshed once every 20 to 30 seconds approximately

        Args:
            station_codes (list): list of WMATA metro station codes for which to retrieve data

        Returns:
            api_response (requests.Response): response object containing json structure
        '''
        station_codes_str = ','.join(station_codes)
        method_url = quote(f'StationPrediction.svc/json/GetPrediction/{station_codes_str}')
        api_response = self.__api_call(method_url)

        return api_response
        


In [158]:
with open('secrets.json', 'r') as f:
    secrets_json_txt = f.read()
    secrets_dict = json.loads(secrets_json_txt)

In [163]:
api_token = secrets_dict['wmata_api_key']
client = WmataClient(api_token)

In [164]:
resp = client.get_prediction(['B08'])

http://api.wmata.com/StationPrediction.svc/json/GetPrediction/B08


In [165]:
resp.json()['Trains']


[{'Car': '8',
  'Destination': 'Glenmont',
  'DestinationCode': 'B11',
  'DestinationName': 'Glenmont',
  'Group': '1',
  'Line': 'RD',
  'LocationCode': 'B08',
  'LocationName': 'Silver Spring',
  'Min': 'BRD'},
 {'Car': '8',
  'Destination': 'Glenmont',
  'DestinationCode': 'B11',
  'DestinationName': 'Glenmont',
  'Group': '1',
  'Line': 'RD',
  'LocationCode': 'B08',
  'LocationName': 'Silver Spring',
  'Min': '4'},
 {'Car': '8',
  'Destination': 'Shady Grove',
  'DestinationCode': None,
  'DestinationName': 'Shady Grove',
  'Group': '2',
  'Line': 'RD',
  'LocationCode': 'B08',
  'LocationName': 'Silver Spring',
  'Min': '4'},
 {'Car': '6',
  'Destination': 'Glenmont',
  'DestinationCode': 'B11',
  'DestinationName': 'Glenmont',
  'Group': '1',
  'Line': 'RD',
  'LocationCode': 'B08',
  'LocationName': 'Silver Spring',
  'Min': '12'},
 {'Car': '8',
  'Destination': 'Shady Grove',
  'DestinationCode': None,
  'DestinationName': 'Shady Grove',
  'Group': '2',
  'Line': 'RD',
  'Loca

In [143]:
class NotOk():
    def __init__(self):
        self.ok = False

In [ ]:
@exponential_backoff
def fail():
    return NotOk()

TypeError: exponential_backoff() got an unexpected keyword argument 'max_calls'

In [145]:
fail()

Exception: Process exceeded retry limit of 3 after waiting 4 seconds between attempts